In [2]:
# Import the libraries
import numpy as np
from tensorflow import keras
from keras import layers

# Convert an integer to a binary double array
def convertIntToBinaryDoubleArray(value):
	result = np.zeros(32)
	for i in range(32):
		result[i] = (value >> i) & 1
	return result

# Create the training data (values from 0 to 100)
nb_data = 1000
training_data = [[], []]
for i in range(nb_data):
	a = np.random.randint(0, 100)
	b = np.random.randint(0, 100)
	c = a + b
	training_data[0].append(np.concatenate((convertIntToBinaryDoubleArray(a), convertIntToBinaryDoubleArray(b))))
	training_data[1].append(convertIntToBinaryDoubleArray(c))

In [3]:
# Create the model
model = keras.Sequential(
	[
		keras.Input(shape = (64, )),
		layers.Dense(48, activation = "sigmoid"),
		layers.Dense(48, activation = "sigmoid"),
		layers.Dense(48, activation = "sigmoid"),
		layers.Dense(32, activation = "sigmoid"),
	]
)

# Print the model summary
model.summary()

# Compile the model with MSE loss, Adam optimizer and accuracy metric
model.compile(
	loss = keras.losses.MeanSquaredError(),
	optimizer = keras.optimizers.SGD(learning_rate = 0.1),
	metrics = ["accuracy"],
)

# Train the model using multi threading
model.fit(
	np.array(training_data[0]),
	np.array(training_data[1]),
	batch_size = 1,
	epochs = 200,
	verbose = 2,
	validation_split = 0.2,
	use_multiprocessing = True,
	workers = 8,
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                3120      
                                                                 
 dense_1 (Dense)             (None, 48)                2352      
                                                                 
 dense_2 (Dense)             (None, 48)                2352      
                                                                 
 dense_3 (Dense)             (None, 32)                1568      
                                                                 
Total params: 9392 (36.69 KB)
Trainable params: 9392 (36.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
800/800 - 1s - loss: 0.0870 - accuracy: 0.0838 - val_loss: 0.0651 - val_accuracy: 0.0100 - 1s/epoch - 2ms/step
Epoch 2/200
800/800 - 1s - loss

In [4]:
# Convert a binary double array to an integer
def convertBinaryDoubleArrayToInt(binary_double_array, start_index):
	result = 0
	for i in range(32):
		result |= int(binary_double_array[start_index + i] + 0.5) << i
	return result

# Test the model on all the training data like in the C code
test_inputs = np.array(training_data[0])
test_expected = np.array(training_data[1])
test_outputs = model.predict(test_inputs, batch_size = len(test_inputs))
nb_errors = 0
for i in range(len(test_inputs)):
	a = convertBinaryDoubleArrayToInt(test_inputs[i], 0)
	b = convertBinaryDoubleArrayToInt(test_inputs[i], 32)
	c = convertBinaryDoubleArrayToInt(test_outputs[i], 0)
	d = convertBinaryDoubleArrayToInt(test_expected[i], 0)
	if c != d:
		nb_errors += 1
		print(f"main(): Error for {a} + {b} = {c} (expected {d})")
print(f"main(): Success rate: {len(test_inputs) - nb_errors}/{len(test_inputs)} ({(len(test_inputs) - nb_errors) / len(test_inputs) * 100.0}%)")

1/1 [==============================] - 0s 111ms/step
main(): Error for 38 + 19 = 91 (expected 57)
main(): Error for 30 + 68 = 91 (expected 98)
main(): Error for 91 + 56 = 91 (expected 147)
main(): Error for 71 + 2 = 91 (expected 73)
main(): Error for 90 + 9 = 91 (expected 99)
main(): Error for 61 + 17 = 91 (expected 78)
main(): Error for 21 + 6 = 91 (expected 27)
main(): Error for 14 + 39 = 91 (expected 53)
main(): Error for 18 + 97 = 91 (expected 115)
main(): Error for 74 + 66 = 91 (expected 140)
main(): Error for 98 + 61 = 91 (expected 159)
main(): Error for 82 + 78 = 91 (expected 160)
main(): Error for 36 + 35 = 91 (expected 71)
main(): Error for 99 + 78 = 91 (expected 177)
main(): Error for 9 + 75 = 91 (expected 84)
main(): Error for 15 + 86 = 91 (expected 101)
main(): Error for 75 + 90 = 91 (expected 165)
main(): Error for 85 + 91 = 91 (expected 176)
main(): Error for 92 + 48 = 91 (expected 140)
main(): Error for 0 + 55 = 91 (expected 55)
main(): Error for 32 + 61 = 91 (expected 9